In [1]:
import numpy as np
import openmc
import openmc.deplete
from pyne import serpent
import re
from openmc.data import zam, gnd_name, atomic_mass
from openmc.deplete.coupled_operator import _get_nuclides_with_data

In [2]:
og_mats = openmc.Materials.from_xml()
fuel = og_mats[0]

In [3]:
# Load OpenMC depletion results
dep_omc = openmc.deplete.Results(filename='openmc-results/depletion_results.h5')

In [4]:
name_dict = dep_omc[0].index_nuc
sorted_names = sorted(name_dict.items(), key=lambda x:x[1])
mat = dep_omc.export_to_materials(0)
mass_percents = []
nuc_mass = []
nuc_names = []
tot_mass = mat[0].get_mass()
serp_spec = []
og_nucs = fuel.get_nuclides()
nuclides_with_data = _get_nuclides_with_data('/home/ooblack/projects/cross-section-libraries/endfb71_h5/cross_sections.xml')
for name, idx in sorted_names:
    if name in nuclides_with_data:
        mass_percent = mat[0].get_mass(name) / tot_mass
        mass_percents.append(mass_percent)
        nuc_mass.append(atomic_mass(name))
        nuc_names.append(name)
        z, a, m = zam(name)
        serp_zam = str(z*1000 + a + (300*int(bool(m)) + 100*m))
        if name in nuclides_with_data:
            serp_zam = ''.join([serp_zam, '.82c'])
        else:
            serp_zam = ''.join([serp_zam, str(m)])
        serp_space = ''.join([" "]*(20 - len(serp_zam)))
        serp_zam = serp_space + serp_zam
        serp_spec.append(serp_zam + "  -" + str(mass_percent) + '\n')
mass_percents = np.array(mass_percents)
nuc_mass = np.array(nuc_mass)
    

/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)


In [5]:
at_percents = (mass_percents/nuc_mass) / np.sum(mass_percents/nuc_mass)

In [6]:
tot_mass

162363299.18199995

In [10]:
og_mats

[Material
 	ID             =	1
 	Name           =	fuel
 	Temperature    =	None
 	Density        =	3.3332642 [g/cm3]
 	S(a,b) Tables  
 	Nuclides       
 	Li7            =	0.12957465539473384 [wo]
 	Be9            =	0.024761094770805 [wo]
 	F19            =	0.5515736451584381 [wo]
 	Th232          =	0.28688745979607144 [wo]
 	U233           =	0.007203144879951567 [wo],
 Material
 	ID             =	2
 	Name           =	graphite
 	Temperature    =	None
 	Density        =	1.843 [g/cm3]
 	S(a,b) Tables  
 	S(a,b)         =	('c_Graphite', 1.0)
 	Nuclides       
 	C0             =	1.0          [wo],
 Material
 	ID             =	3
 	Name           =	hastelloyN
 	Temperature    =	None
 	Density        =	8.671 [g/cm3]
 	S(a,b) Tables  
 	Nuclides       
 	B10            =	1.835549587507326e-06 [wo]
 	B11            =	8.164450412492673e-06 [wo]
 	Al27           =	0.001        [wo]
 	Si28           =	0.0009187351728295046 [wo]
 	Si29           =	4.831750292845674e-05 [wo]
 	Si30           =	3.2947

In [9]:
dict(zip(nuc_names, mass_percents))['U233']

0.007208481408313772

In [41]:
serp_spec = []
for name, tup in mat[2].get_nuclide_densities().items():
    name, mass_percent, percent_type = tup
    z, a, m = zam(name)
    serp_zam = str(z*1000 + a + (300*int(bool(m)) + 100*m))
    if name in nuclides_with_data:
        serp_zam = ''.join([serp_zam, '.82c'])
    else:
        serp_zam = ''.join([serp_zam, str(m)])
    serp_space = ''.join([" "]*(20 - len(serp_zam)))
    serp_zam = serp_space + serp_zam
    serp_spec.append(serp_zam + "  -" + str(mass_percent) + '\n')

In [8]:
with open('serp_spec.txt', 'w') as f:
    f.writelines(serp_spec)